<a href="https://colab.research.google.com/github/psbhargava/100daysofnlp/blob/master/Day22__LatentDirichletAllocation(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kaggle into Colab

In [0]:
 ! pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

## The Data

In [6]:
! kaggle datasets download -d therohk/million-headlines

 84% 17.0M/20.2M [00:00<00:00, 21.6MB/s]
100% 20.2M/20.2M [00:00<00:00, 29.7MB/s]


In [7]:
! unzip /content/million-headlines.zip

Archive:  /content/million-headlines.zip
  inflating: abcnews-date-text.csv   


In [0]:
import numpy as np
import pandas as pd

np.random.seed(2018)

In [0]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [10]:
len(documents)

1186018

In [11]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Data Pre-processing

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

np.random.seed(2018)

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [15]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
           
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [0]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    

def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result.append(lemmatize_stemming(token))
  return result


In [17]:

doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')

words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)

print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [18]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [19]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]

In [23]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]],bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3567 ("compulsori") appears 1 time.
Word 3568 ("ratepay") appears 1 time.


In [24]:
from gensim import corpora, models
from pprint import pprint

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [27]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.030*"charg" + 0.027*"court" + 0.025*"murder" + 0.020*"woman" + 0.019*"news" + 0.018*"face" + 0.017*"alleg" + 0.016*"jail" + 0.016*"polic" + 0.015*"accus"
Topic: 1 
Words: 0.030*"australia" + 0.020*"home" + 0.019*"queensland" + 0.016*"world" + 0.014*"record" + 0.013*"rise" + 0.012*"price" + 0.012*"break" + 0.009*"scott" + 0.009*"high"
Topic: 2 
Words: 0.040*"polic" + 0.024*"melbourn" + 0.019*"live" + 0.017*"death" + 0.016*"brisban" + 0.015*"royal" + 0.014*"bushfir" + 0.014*"investig" + 0.013*"interview" + 0.012*"commiss"
Topic: 3 
Words: 0.015*"chang" + 0.015*"govern" + 0.014*"say" + 0.013*"tasmanian" + 0.012*"countri" + 0.012*"minist" + 0.012*"state" + 0.011*"deal" + 0.010*"talk" + 0.009*"climat"
Topic: 4 
Words: 0.019*"nation" + 0.013*"farmer" + 0.013*"plan" + 0.012*"rural" + 0.012*"health" + 0.012*"fund" + 0.012*"council" + 0.012*"indigen" + 0.010*"govern" + 0.010*"work"
Topic: 5 
Words: 0.039*"trump" + 0.019*"elect" + 0.015*"tasmania" + 0.013*"fight" + 0.012*"spea

In [0]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [30]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"tuesday" + 0.007*"care" + 0.007*"korea" + 0.007*"morrison" + 0.006*"scott" + 0.006*"john" + 0.006*"age" + 0.006*"say" + 0.005*"north" + 0.005*"hong"
Topic: 1 Word: 0.027*"trump" + 0.009*"climat" + 0.008*"grandstand" + 0.006*"june" + 0.005*"brief" + 0.005*"chang" + 0.004*"creek" + 0.004*"univers" + 0.004*"johnson" + 0.004*"financ"
Topic: 2 Word: 0.009*"guilti" + 0.008*"turnbul" + 0.007*"elect" + 0.007*"sexual" + 0.006*"plead" + 0.006*"liber" + 0.006*"abus" + 0.006*"assault" + 0.005*"commiss" + 0.005*"insid"
Topic: 3 Word: 0.014*"news" + 0.012*"rural" + 0.011*"market" + 0.009*"govern" + 0.007*"farmer" + 0.006*"price" + 0.006*"nation" + 0.006*"drought" + 0.005*"wednesday" + 0.005*"fund"
Topic: 4 Word: 0.008*"monday" + 0.007*"cattl" + 0.006*"septemb" + 0.006*"festiv" + 0.006*"outback" + 0.005*"cancer" + 0.004*"firefight" + 0.004*"music" + 0.004*"hobart" + 0.004*"solar"
Topic: 5 Word: 0.012*"murder" + 0.012*"drum" + 0.011*"charg" + 0.008*"court" + 0.008*"woman" + 0.008

In [31]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [32]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5028274655342102	 
Topic: 0.019*"nation" + 0.013*"farmer" + 0.013*"plan" + 0.012*"rural" + 0.012*"health" + 0.012*"fund" + 0.012*"council" + 0.012*"indigen" + 0.010*"govern" + 0.010*"work"

Score: 0.26249995827674866	 
Topic: 0.039*"trump" + 0.019*"elect" + 0.015*"tasmania" + 0.013*"fight" + 0.012*"speak" + 0.012*"australia" + 0.011*"game" + 0.010*"world" + 0.009*"want" + 0.009*"liber"

Score: 0.14717194437980652	 
Topic: 0.022*"call" + 0.022*"canberra" + 0.016*"busi" + 0.014*"victoria" + 0.014*"farm" + 0.013*"drum" + 0.012*"bank" + 0.010*"turn" + 0.010*"labor" + 0.010*"parliament"

Score: 0.012500617653131485	 
Topic: 0.015*"chang" + 0.015*"govern" + 0.014*"say" + 0.013*"tasmanian" + 0.012*"countri" + 0.012*"minist" + 0.012*"state" + 0.011*"deal" + 0.010*"talk" + 0.009*"climat"

Score: 0.012500000186264515	 
Topic: 0.030*"charg" + 0.027*"court" + 0.025*"murder" + 0.020*"woman" + 0.019*"news" + 0.018*"face" + 0.017*"alleg" + 0.016*"jail" + 0.016*"polic" + 0.015*"accus"

Score

In [33]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5561744570732117	 
Topic: 0.014*"news" + 0.012*"rural" + 0.011*"market" + 0.009*"govern" + 0.007*"farmer" + 0.006*"price" + 0.006*"nation" + 0.006*"drought" + 0.005*"wednesday" + 0.005*"fund"

Score: 0.19375298917293549	 
Topic: 0.010*"stori" + 0.008*"live" + 0.007*"australia" + 0.006*"leagu" + 0.006*"rugbi" + 0.006*"zealand" + 0.005*"august" + 0.005*"david" + 0.005*"world" + 0.004*"rise"

Score: 0.16256174445152283	 
Topic: 0.012*"donald" + 0.010*"interview" + 0.008*"weather" + 0.008*"street" + 0.007*"queensland" + 0.007*"island" + 0.006*"christma" + 0.006*"cyclon" + 0.006*"andrew" + 0.005*"violenc"

Score: 0.01250272337347269	 
Topic: 0.009*"guilti" + 0.008*"turnbul" + 0.007*"elect" + 0.007*"sexual" + 0.006*"plead" + 0.006*"liber" + 0.006*"abus" + 0.006*"assault" + 0.005*"commiss" + 0.005*"insid"

Score: 0.012502199970185757	 
Topic: 0.009*"tuesday" + 0.007*"care" + 0.007*"korea" + 0.007*"morrison" + 0.006*"scott" + 0.006*"john" + 0.006*"age" + 0.006*"say" + 0.005*"north" +

In [34]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3500000238418579	 Topic: 0.015*"chang" + 0.015*"govern" + 0.014*"say" + 0.013*"tasmanian" + 0.012*"countri"
Score: 0.3499999940395355	 Topic: 0.039*"trump" + 0.019*"elect" + 0.015*"tasmania" + 0.013*"fight" + 0.012*"speak"
Score: 0.18333329260349274	 Topic: 0.030*"charg" + 0.027*"court" + 0.025*"murder" + 0.020*"woman" + 0.019*"news"
Score: 0.01666666753590107	 Topic: 0.030*"australia" + 0.020*"home" + 0.019*"queensland" + 0.016*"world" + 0.014*"record"
Score: 0.01666666753590107	 Topic: 0.040*"polic" + 0.024*"melbourn" + 0.019*"live" + 0.017*"death" + 0.016*"brisban"
Score: 0.01666666753590107	 Topic: 0.019*"nation" + 0.013*"farmer" + 0.013*"plan" + 0.012*"rural" + 0.012*"health"
Score: 0.01666666753590107	 Topic: 0.025*"south" + 0.022*"north" + 0.021*"coast" + 0.016*"adelaid" + 0.015*"power"
Score: 0.01666666753590107	 Topic: 0.025*"kill" + 0.025*"attack" + 0.023*"report" + 0.022*"donald" + 0.019*"die"
Score: 0.01666666753590107	 Topic: 0.022*"call" + 0.022*"canberra" + 0.01